# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [5]:
# Corrected file path
file_path = "/Users/janlelie/Desktop/Git Hub Repositories/python-api-challenge/WeatherPy/output_data/cities.csv"

# Load the CSV file
city_data_df = pd.read_csv(file_path)

# Display the first few rows to verify
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,tiksi,71.6872,128.8694,-21.83,84,57,7.13,RU,1732514949
1,1,jamestown,42.0970,-79.2353,3.34,95,100,2.57,US,1732514755
2,2,adamstown,-25.0660,-130.1015,22.25,71,49,7.53,PN,1732514953
3,3,yellowknife,62.4560,-114.3525,-8.66,96,96,4.47,CA,1732514954
4,4,udachny,66.4167,112.4000,-24.51,80,68,6.36,RU,1732514955


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [6]:
import hvplot.pandas
import geoviews as gv
from geoviews import opts
import holoviews as hv

# Enable Holoviews extension for bokeh
hv.extension('bokeh')

# Configure the map plot
city_map = city_data_df.hvplot.points(
    'Lng', 'Lat',           # Longitude and Latitude for plotting
    geo=True,               # Enable geospatial plotting
    size='Humidity',        # Size of the points based on humidity
    color='Humidity',       # Color the points by humidity
    alpha=0.6,              # Add transparency for better visualization
    tiles='OSM',            # OpenStreetMap tiles for the base map
    frame_width=800,        # Map width
    frame_height=600,       # Map height
    title="City Humidity Map"
)

# Display the map
city_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [7]:
# Narrow down cities that fit criteria
ideal_cities_df = city_data_df[
    (city_data_df["Max Temp"] > 21) & 
    (city_data_df["Max Temp"] < 27) & 
    (city_data_df["Wind Speed"] < 4.5) & 
    (city_data_df["Cloudiness"] == 0)
]

# Drop any rows with null values
ideal_cities_df = ideal_cities_df.dropna()

# Display sample data
ideal_cities_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
28,28,kailua-kona,19.6406,-155.9956,23.98,79,0,1.54,US,1732514991
63,63,lihue,21.9789,-159.3672,21.77,92,0,3.09,US,1732515049
100,100,cozumel,20.5083,-86.9458,23.81,66,0,3.82,MX,1732515004
163,163,wailua homesteads,22.0669,-159.3780,21.44,89,0,3.09,US,1732515202
168,168,san isidro de curuguaty,-24.5167,-55.7000,22.15,75,0,1.54,PY,1732515211


### Step 3: Create a new DataFrame called `hotel_df`.

In [8]:
# Use the Pandas copy function to create DataFrame called hotel_df
# Include city, country, coordinates, and humidity
hotel_df = ideal_cities_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
28,kailua-kona,US,19.6406,-155.9956,79,
63,lihue,US,21.9789,-159.3672,92,
100,cozumel,MX,20.5083,-86.9458,66,
163,wailua homesteads,US,22.0669,-159.3780,89,
168,san isidro de curuguaty,PY,-24.5167,-55.7000,75,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [9]:
import time

# Set parameters to search for a hotel
radius = 10000  # 10,000 meters (10 km)
params = {
    "categories": "accommodation.hotel",  # Search for hotels
    "limit": 1,  # Retrieve only the first result
    "apiKey": geoapify_key  # Your Geoapify API key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude and longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found"
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

    # Pause to avoid exceeding the rate limit
    time.sleep(1)

# Display sample data
hotel_df

Starting hotel search
kailua-kona - nearest hotel: PACIFIC 19 Kona
lihue - nearest hotel: Kauai Palms
cozumel - nearest hotel: Hotel Isla Esmeralda
wailua homesteads - nearest hotel: Hilton Garden Inn Kauai Wailua Bay
san isidro de curuguaty - nearest hotel: Hospedaje El Sol
kapa'a - nearest hotel: Pono Kai Resort
rojhan - nearest hotel: No hotel found
zouerate - nearest hotel: فندق تيرس زمور
ataq - nearest hotel: الشارقة بلازا
timbuktu - nearest hotel: Hotel La Maison
jaisalmer - nearest hotel: Jeet Mahal
mukalla - nearest hotel: فندق الجابري


,City,Country,Lat,Lng,Humidity,Hotel Name
28,kailua-kona,US,19.6406,-155.9956,79,PACIFIC 19 Kona
63,lihue,US,21.9789,-159.3672,92,Kauai Palms
100,cozumel,MX,20.5083,-86.9458,66,Hotel Isla Esmeralda
163,wailua homesteads,US,22.0669,-159.3780,89,Hilton Garden Inn Kauai Wailua Bay
168,san isidro de curuguaty,PY,-24.5167,-55.7000,75,Hospedaje El Sol
193,kapa'a,US,22.0752,-159.3190,90,Pono Kai Resort
265,rojhan,PK,28.6868,69.9534,21,No hotel found
291,zouerate,MR,22.7187,-12.4521,17,فندق تيرس زمور
410,ataq,YE,14.5377,46.8319,43,الشارقة بلازا
441,timbuktu,ML,16.7735,-3.0074,17,Hotel La Maison


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [10]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    'Lng', 'Lat',               # Longitude and Latitude for plotting
    geo=True,                   # Enable geospatial plotting
    size='Humidity',            # Size of the points based on humidity
    color='Humidity',           # Color the points by humidity
    alpha=0.7,                  # Add transparency for better visualization
    tiles='OSM',                # OpenStreetMap tiles for the base map
    frame_width=800,            # Map width
    frame_height=600,           # Map height
    hover_cols=['Hotel Name', 'Country'],  # Add hover info for hotel name and country
    title="Hotel Map with Humidity"
)

# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,Hotel Name,Country)